
# Monte Carlo Supply Chain — Poudre ABC (24 mois, mensuel)
## Variante recommandée : **Option B (réaliste)** — volume annuel tiré **par année**

### Pourquoi Option B ?
En pratique, une usine pilote a un **plan annuel** (budget, capacité, contrats) qui est **révisé chaque année**.
Donc il est réaliste de :
- tirer une production annuelle **Année 1** : \(P_{ann,1} \sim U(11000,15000)\),
- tirer une production annuelle **Année 2** : \(P_{ann,2} \sim U(11000,15000)\),
- construire `plan_prod` (24 mois) en concaténant les 2 plans saisonniers.

La simulation Monte Carlo répète ce processus pour produire une distribution de risques (rupture/surstock).

> ⚠️ Les paramètres `sigma_prod`, lead times (min/mode/max), stocks de sécurité, etc. sont des **valeurs de départ** modifiables.


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, List, Tuple


In [ ]:
@dataclass
class MaterialPolicy:
    name: str
    x: float                   # fraction massique
    review_period: int         # R_i (mois)
    ss_months: float           # stock de sécurité (mois de conso)
    lt_min: float              # lead time min (mois)
    lt_mode: float             # lead time mode (mois)
    lt_max: float              # lead time max (mois)
    stock_max_months: float    # seuil surstock (mois de conso) - optionnel


@dataclass
class SimulationConfig:
    horizon_months: int = 24
    sigma_prod: float = 0.10     # <-- placeholder: variabilité mensuelle de production autour du plan
    n_iter: int = 3000           # <-- itérations Monte Carlo
    seed: int = 42

    # Incertitude sur la production annuelle (Uniforme)
    annual_min: float = 11000.0
    annual_max: float = 15000.0

    # Retards exceptionnels (pour matières externes)
    p_delay_1: float = 0.15
    p_delay_2: float = 0.05
    delay_1: int = 1
    delay_2: int = 2


In [ ]:
def seasonal_plan_12_months(annual_production: float) -> np.ndarray:
    """
    Construit 12 mois de production planifiée à partir de la saisonnalité fournie :
      - Hiver 23%, Printemps 24%, Été 26%, Automne 27%
    Chaque saison = 3 mois.
    """
    p = {"Hiver": 0.23, "Printemps": 0.24, "Ete": 0.26, "Automne": 0.27}

    winter_m = (p["Hiver"] * annual_production) / 3.0
    spring_m = (p["Printemps"] * annual_production) / 3.0
    summer_m = (p["Ete"] * annual_production) / 3.0
    autumn_m = (p["Automne"] * annual_production) / 3.0

    year_pattern = [winter_m]*3 + [spring_m]*3 + [summer_m]*3 + [autumn_m]*3
    return np.array(year_pattern, dtype=float)


def sample_triangular_int(rng: np.random.Generator, a: float, c: float, b: float) -> int:
    """Triangulaire puis arrondi au supérieur => délai entier en mois."""
    val = rng.triangular(a, c, b)
    return int(math.ceil(val))


def sample_delay(rng: np.random.Generator, cfg: SimulationConfig) -> int:
    """Retard additionnel ΔLT pour matières externes."""
    u = rng.random()
    if u < cfg.p_delay_2:
        return cfg.delay_2
    elif u < cfg.p_delay_2 + cfg.p_delay_1:
        return cfg.delay_1
    return 0


In [ ]:
def simulate_one_path_with_history(
    rng: np.random.Generator,
    cfg: SimulationConfig,
    materials: List[MaterialPolicy],
    plan_prod: np.ndarray,
    initial_stocks: Dict[str, float],
    target_levels: Dict[str, float],
    internal_materials: set,
) -> Tuple[Dict[str, float], Dict[str, np.ndarray]]:
    """
    Simule une trajectoire sur T mois avec politique (R,S), et retourne:
      - stock_min[name] : stock minimum sur l'horizon (utile pour le risque rupture)
      - stock_hist[name][t] : stock fin de mois t
    """
    T = cfg.horizon_months
    receipts = {m.name: np.zeros(T, dtype=float) for m in materials}
    stock = {m.name: float(initial_stocks[m.name]) for m in materials}
    stock_hist = {m.name: np.zeros(T, dtype=float) for m in materials}

    stock_min = {m.name: stock[m.name] for m in materials}

    for t in range(T):
        # 1) Production réelle aléatoire autour du plan mensuel
        eps = rng.normal(0.0, cfg.sigma_prod)
        prod_t = max(0.0, plan_prod[t] * (1.0 + eps))

        # 2) Réceptions du mois t
        for m in materials:
            stock[m.name] += receipts[m.name][t]

        # 3) Consommation + historisation
        for m in materials:
            stock[m.name] -= m.x * prod_t
            stock_hist[m.name][t] = stock[m.name]
            stock_min[m.name] = min(stock_min[m.name], stock[m.name])

        # 4) Commandes (R,S) si mois de revue
        for m in materials:
            if (t % m.review_period) == 0:
                pipeline = receipts[m.name][t:].sum()
                inv_position = stock[m.name] + pipeline
                q = max(0.0, target_levels[m.name] - inv_position)

                if q > 0:
                    lt = sample_triangular_int(rng, m.lt_min, m.lt_mode, m.lt_max)

                    # Retards exceptionnels: désactivés pour les matières internes (MAP)
                    if m.name not in internal_materials:
                        lt += sample_delay(rng, cfg)

                    arrival_t = t + lt
                    if arrival_t < T:
                        receipts[m.name][arrival_t] += q

    return stock_min, stock_hist



## Paramètres matières (à ajuster)
- `review_period` : 1 = mensuel, 3 = trimestriel
- `ss_months` : stock de sécurité en mois de consommation
- lead times `lt_min/lt_mode/lt_max` : valeurs de départ à remplacer


In [ ]:
cfg = SimulationConfig(
    horizon_months=24,
    sigma_prod=0.10,
    n_iter=3000,
    seed=42,
    annual_min=11000.0,
    annual_max=15000.0,
)

materials = [
    MaterialPolicy("MAP",   x=0.40,  review_period=1, ss_months=0.5, lt_min=0.0, lt_mode=0.0, lt_max=1.0, stock_max_months=2.0),
    MaterialPolicy("SA",    x=0.565, review_period=1, ss_months=1.0, lt_min=0.0, lt_mode=0.5, lt_max=1.0, stock_max_months=3.0),
    MaterialPolicy("Mica",  x=0.02,  review_period=3, ss_months=2.0, lt_min=1.0, lt_mode=2.0, lt_max=3.0, stock_max_months=6.0),
    MaterialPolicy("Silice",x=0.005, review_period=3, ss_months=3.0, lt_min=1.0, lt_mode=2.0, lt_max=4.0, stock_max_months=12.0),
    MaterialPolicy("Huile", x=0.01,  review_period=3, ss_months=2.0, lt_min=1.0, lt_mode=2.0, lt_max=3.0, stock_max_months=12.0),
]

internal_materials = {"MAP"}

rng = np.random.default_rng(cfg.seed)

print("Config:", cfg)
print("Matières:", [m.name for m in materials])



## Monte Carlo — volume annuel incertain (Uniforme) **par année**

Pour chaque itération :
1. Tirer \(P_{ann,1} \sim U(11000,15000)\) et \(P_{ann,2} \sim U(11000,15000)\).
2. Construire `plan_prod` sur 24 mois (12 mois année 1 + 12 mois année 2).
3. Dimensionner \(S^*\) et \(S_0\) de façon cohérente avec la consommation moyenne du plan.
4. Simuler la trajectoire (stocks, commandes, délais, retards).


In [ ]:
stock_paths = {m.name: np.zeros((cfg.n_iter, cfg.horizon_months), dtype=float) for m in materials}
stock_mins = {m.name: np.zeros(cfg.n_iter, dtype=float) for m in materials}

annual_draws_year1 = np.zeros(cfg.n_iter, dtype=float)
annual_draws_year2 = np.zeros(cfg.n_iter, dtype=float)

example_hist = None
example_plan = None
example_annuals = None

for k in range(cfg.n_iter):
    # 1) Tirage du volume annuel par année (Option B)
    ann1 = rng.uniform(cfg.annual_min, cfg.annual_max)
    ann2 = rng.uniform(cfg.annual_min, cfg.annual_max)
    annual_draws_year1[k] = ann1
    annual_draws_year2[k] = ann2

    # 2) Construction du plan 24 mois
    plan1 = seasonal_plan_12_months(ann1)
    plan2 = seasonal_plan_12_months(ann2)
    plan_prod = np.concatenate([plan1, plan2])

    # 3) Dimensionnement (placeholders) des niveaux cibles S* et stocks initiaux S0
    avg_prod = plan_prod.mean()
    avg_cons = {m.name: m.x * avg_prod for m in materials}

    target_levels = {}
    initial_stocks = {}
    for m in materials:
        target_levels[m.name] = (m.lt_mode + m.review_period + m.ss_months) * avg_cons[m.name]
        initial_stocks[m.name] = (m.lt_mode + m.ss_months) * avg_cons[m.name]

    # 4) Simulation d'une trajectoire
    stock_min_k, hist_k = simulate_one_path_with_history(
        rng, cfg, materials, plan_prod, initial_stocks, target_levels, internal_materials
    )

    for m in materials:
        stock_paths[m.name][k, :] = hist_k[m.name]
        stock_mins[m.name][k] = stock_min_k[m.name]

    # Sauvegarde d'un exemple (k==0) pour illustration
    if k == 0:
        example_hist = hist_k
        example_plan = plan_prod
        example_annuals = (ann1, ann2)

months = np.arange(1, cfg.horizon_months + 1)

print("Monte Carlo terminé.")
print(f"Itérations={cfg.n_iter} | sigma_prod={cfg.sigma_prod:.0%}")
print(f"Exemple: Année1={example_annuals[0]:.0f} t/an | Année2={example_annuals[1]:.0f} t/an")


## Visualisation du plan de production (exemple) — 1 figure

In [ ]:
plt.figure()
plt.plot(months, example_plan)
plt.xlabel("Mois")
plt.ylabel("Production planifiée (t)")
plt.title("Plan de production (exemple) sur 24 mois — Option B (2 années tirées)")
plt.show()


## Plots — Trajectoire exemple (1 figure par matière)

In [ ]:
for m in materials:
    plt.figure()
    plt.plot(months, example_hist[m.name])
    plt.axhline(0, linestyle="--")
    plt.xlabel("Mois")
    plt.ylabel("Stock (t)")
    plt.title(f"Trajectoire exemple - Stock {m.name}")
    plt.show()


## Plots — Bandes Monte Carlo P5/P50/P95 (1 figure par matière)

In [ ]:
for m in materials:
    p5  = np.percentile(stock_paths[m.name], 5, axis=0)
    p50 = np.percentile(stock_paths[m.name], 50, axis=0)
    p95 = np.percentile(stock_paths[m.name], 95, axis=0)

    plt.figure()
    plt.plot(months, p50)
    plt.fill_between(months, p5, p95, alpha=0.2)
    plt.axhline(0, linestyle="--")
    plt.xlabel("Mois")
    plt.ylabel("Stock (t)")
    plt.title(f"Monte Carlo - {m.name} (P5/P50/P95) — Option B")
    plt.show()


## Histogrammes — Stock minimum sur 24 mois (1 figure par matière)

In [ ]:
for m in materials:
    plt.figure()
    plt.hist(stock_mins[m.name], bins=40)
    plt.axvline(0, linestyle="--")
    plt.xlabel("Stock minimum sur 24 mois (t)")
    plt.ylabel("Fréquence")
    plt.title(f"Distribution du stock minimum - {m.name}")
    plt.show()


## KPI synthèse (texte)

In [ ]:
for m in materials:
    name = m.name
    p_any_stockout = float(np.mean(stock_mins[name] < 0))
    p5_min, p50_min, p95_min = np.percentile(stock_mins[name], [5, 50, 95])
    max_per_run = stock_paths[name].max(axis=1)
    p95_max = float(np.percentile(max_per_run, 95))

    print(f"[{name}]")
    print(f"  P(au moins une rupture sur 24 mois) = {p_any_stockout:.1%}")
    print(f"  Stock minimum (P5/P50/P95) = {p5_min:.1f} / {p50_min:.1f} / {p95_min:.1f} t")
    print(f"  Stock maximum (P95) = {p95_max:.1f} t")
    print("")
